# SC1015 Lab 8
## Introduction
Modern enterprises face decision-making challenges of exponential complexity [1]. Three key drivers include: (1) the big data revolution increasing decision variables significantly, (2) interdependent real-world systems and (3) objectives with competing priorities [2, 3].
The team aims to explore combinatorial optimization through a simplified version of the Traffic Jam game. In this environment, an AI agent formulates an optimal path to a goal that minimizes the total number of moves. This is done by mapping board states to moves taken and minimizing vehicle movement.
These same principles scale remarkably to global shipping optimization [4], where:
•	Each port call represents a "move" in the Traffic Jam analogy.
•	Fuel costs and emissions act as the "reward function" to optimize.
•	Weather patterns and port schedules create the "constraint network."
With AI-driven optimization algorithms, shipping companies can find the most efficient routing strategies to improve operational costs, reduce emissions, and ensure timely deliveries. This demonstrates how principles learned in simpler environments can be translated to real-world applications.

## Members
Name: Kee Chong Wei |  Matriculation Number: U2320846D <br>
Name: Jiang Zong Zhe | Matriculation Number: U2322460F <br>
Name: Chew Jin Cheng | Matriculation Number: U2321537J <br>

## References
[1] Berutich Lindquist, J. M. (2017). Robust optimization of algorithmic trading systems. 
[2] Mukelabai, M., Nešić, D., Maro, S., Berger, T., & Steghöfer, J. P. (2018, September). Tackling combinatorial explosion: a study of industrial needs and practices for analyzing highly configurable systems. In Proceedings of the 33rd acm/ieee international conference on automated software engineering (pp. 155-166). 
[3] Surana, A., Kumara*, S., Greaves, M., & Raghavan, U. N. (2005). Supply-chain networks: a complex adaptive systems perspective. International Journal of Production Research, 43(20), 4235-4265. 
[4] Walther, L., Rizvanolli, A., Wendebourg, M., & Jahn, C. (2016). Modeling and optimization algorithms in ship weather routing. International journal of e-navigation and maritime economy, 4, 31-45.


In [29]:
import numpy as np
import matplotlib.pyplot as plt
from seaborn import heatmap
import networkx as nx
import queue

In [ ]:
# begin by visualising problem space

def printProblemState(mazeGrid):
    ''' Display the maze corresponding to a binary grid
        Input : 2D NumPy array with 'W', 'O','T','G','E' as elements
        Output : Simple print of the corresponding maze
    '''
    (height, width) = mazeGrid.shape
    
    print()
    for i in range(height):
        for j in range(width):
            if mazeGrid[i,j] == 'W': # walls
                print("\u2b1b", end = " ")   # use some other character if the unicode does not print properly
            elif mazeGrid[i,j] == ' ': # empty spaces
                print("\u2b1c", end = " ")
            elif mazeGrid[i,j] == 'G': # goal
                print("\u2690", end=" ")
            elif mazeGrid[i,j] == 'O': # obstacles
                print("\u2780", end= " ")
            elif mazeGrid[i,j] == 'T': # target vehicle
                print("\u26f4", end=" ")
        print()


link to unicode symbols: https://symbl.cc/en/unicode-table/#miscellaneous-symbols-and-arrows

In [57]:
maze = [['W','W','W','W'],
        ['T','O','O','G'],
        ['W','O',' ',' '],
        ['W','W','W','W']]

mazeTuple = (('W','W','W','W'),
        ('T','O','O','G'),
        ('W','O',' ',' '),
        ('W','W','W','W'))
# Convert to a NumPy array
maze = np.array(maze)

# Print the maze using helper functions
print("Maze of dimensions", maze.shape)
printProblemState(maze)

Maze of dimensions (4, 4)

⬛ ⬛ ⬛ ⬛ 
⛴ ➀ ➀ ⚐ 
⬛ ➀ 
⬛ ⬛ ⬛ ⬛ 


In [60]:
from collections import deque

# Set goal_position once, independent of the grid.
goal_position = (1, 3)  # based on your grid


class Node:
    def __init__(self, grid, move=None, parent=None):
        self.grid = grid          # grid is a tuple of tuples representing the board
        self.move = move          # a tuple: (piece, from_position, to_position)
        self.parent = parent      # pointer to the previous Node (None for the root)

def find_positions(grid, piece):
    positions = []
    for i, row in enumerate(grid):
        for j, cell in enumerate(row):
            if cell == piece:
                positions.append((i, j))
    return positions

def get_target_position(grid):
    positions = find_positions(grid, 'T')
    return positions[0] if positions else None

def get_goal_position(grid):
    positions = find_positions(grid, 'G')
    return positions[0] if positions else None

def manhattan_distance(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def is_goal(grid):
    # Goal is reached when T occupies the same cell as G.
    return get_target_position(grid) == goal_position

def move_piece(grid, pos, direction):
    rows, cols = len(grid), len(grid[0])
    grid_list = [list(row) for row in grid]
    x, y = pos
    dx, dy = direction
    new_x, new_y = x + dx, y + dy

    # Check boundaries.
    if not (0 <= new_x < rows and 0 <= new_y < cols):
        return None

    target_piece = grid_list[x][y]
    destination = grid_list[new_x][new_y]

    if target_piece == 'T':
        # Allow T to move into an empty space or the goal.
        if destination in [' ', 'G']:
            grid_list[new_x][new_y] = 'T'
            # Restore the goal if T moves away from it.
            grid_list[x][y] = 'G' if grid_list[x][y] == 'T' and (x, y) == get_goal_position(grid) else ' '
        else:
            return None
    else:
        # For obstacles, allow only movement into empty space.
        if destination == ' ':
            grid_list[new_x][new_y] = target_piece
            grid_list[x][y] = ' '
        else:
            return None

    return tuple(tuple(row) for row in grid_list)

def expand_node(node):
    children = []
    grid = node.grid
    t_pos = get_target_position(grid)
    g_pos = get_goal_position(grid)
    
    if not t_pos or not g_pos:
        return children

    # Directions: up, down, left, right.
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    current_distance = manhattan_distance(t_pos, g_pos)

    # First try moves for T that reduce the Manhattan distance.
    for d in directions:
        new_grid = move_piece(grid, t_pos, d)
        if new_grid:
            new_t_pos = get_target_position(new_grid)
            new_distance = manhattan_distance(new_t_pos, g_pos)
            if new_distance < current_distance:
                move = ('T', t_pos, (t_pos[0] + d[0], t_pos[1] + d[1]))
                children.append(Node(new_grid, move, node))

    # If T cannot move closer, then expand moves for obstacles.
    if not children:
        obstacles = find_positions(grid, 'O')
        for pos in obstacles:
            for d in directions:
                new_grid = move_piece(grid, pos, d)
                if new_grid:
                    move = ('O', pos, (pos[0] + d[0], pos[1] + d[1]))
                    children.append(Node(new_grid, move, node))
    return children

def search_solution(initial_grid):
    root = Node(initial_grid)
    frontier = deque([root])
    visited = set([str(initial_grid)])
    
    while frontier:
        current_node = frontier.popleft()
        if is_goal(current_node.grid):
            return current_node  # Found goal; return the node to reconstruct the path.
        
        for child in expand_node(current_node):
            state_str = str(child.grid)
            if state_str not in visited:
                visited.add(state_str)
                frontier.append(child)
    return None  # No solution found.

def get_solution_path(node):
    path = []
    while node:
        path.append(node)
        node = node.parent
    path.reverse()
    return path


In [61]:
solution_node = search_solution(mazeTuple)
if solution_node:
    path = get_solution_path(solution_node)
    for step in path:
        print("Move:", step.move)
        for row in step.grid:
            print(" ".join(row))
        print("-----")
else:
    print("No solution found.")

Move: None
W W W W
T O O G
W O    
W W W W
-----
Move: ('O', (1, 2), (2, 2))
W W W W
T O   G
W O O  
W W W W
-----
Move: ('O', (1, 1), (1, 2))
W W W W
T   O G
W O O  
W W W W
-----
Move: ('T', (1, 0), (1, 1))
W W W W
  T O G
W O O  
W W W W
-----
Move: ('O', (2, 2), (2, 3))
W W W W
  T O G
W O   O
W W W W
-----
Move: ('O', (1, 2), (2, 2))
W W W W
  T   G
W O O O
W W W W
-----
Move: ('T', (1, 1), (1, 2))
W W W W
    T G
W O O O
W W W W
-----
Move: ('T', (1, 2), (1, 3))
W W W W
      T
W O O O
W W W W
-----
